# Adding information to BdHNe table

One of these days I published the online version of ICRANet Binary-driven Hyper-Novae table of Gamma-Ray Bursts (GRB) at the (BSDC/VO)[http://vo.bsdc.icranet.org] website; the table: http://vo.bsdc.icranet.org/bdhne/q/web/form.
The (current) first version of the table contains exactly what is being published in the article (Ruffini et al, in preparation), which is:

* GRB (id): the standard name for GRBs (*YYMMDD*[A-Z]);
* Designation: as proposed by the authors of the article;
* z: redshift;
* E_iso: the energy computedby the authors after their model;
* Instrument: observatory from where data was used;
* GCN: the GCN corresponding to the event.

What I'm going to do now is to add position (Right Ascension and Declination) and trigger time of those events.
To do so, I should retrieve such information from online services/tables and join them with the current table.

Swift GRBs data will be used to get such information (position and trigger time). 
(*"--Why Swift? --Because I had to choose one; next time will be Fermi."*)
After a search for which website could give me better the information, the [Mission's American website][NASAGRB].
Through their service I can easily automate the query, retrieve and parsing process, plus they provide all the relevant information I am looking for (almost) all GRBs.
For the records, below is the list of websites I looked into; The list was kindly provided by my colleaghe Milos Kovacevic:

* http://www.swift.ac.uk/xrt_positions/
* https://heasarc.gsfc.nasa.gov/W3Browse/fermi/fermigbrst.html
* https://heasarc.gsfc.nasa.gov/W3Browse/all/batsegrb.html
* http://www.astro.caltech.edu/grbox/grbox.php
* (**Chosen**) https://swift.gsfc.nasa.gov/archive/grb_table/

Data and software we already have (properly working/formatted):

* The current BdHNe table is here copied as filename `ibdhne.csv`;
* The Perl script to retrieve the table from [Swift GRB service][NASAGRB], called `SwiftGRBsNASA.pl`;
 * We will execute the script to generate an up-to-date table of GRBs named `SwiftGRBsNASA_table.csv`.

[NASAGRB]: https://swift.gsfc.nasa.gov/archive/grb_table/

In [11]:
import pandas
pandas.set_option('display.max_columns',None)

## The IBdHNe table

In [3]:
ibd = pandas.read_csv('data.csv',sep=';')
print ibd

         GRB     designation      z   E_iso instrument    GCN
0    050315A  IBdHNe 050315A  1.950    8.32      Swift   3099
1    050318A  IBdHNe 050318A  1.440    3.70      Swift   3134
2    050319A  IBdHNe 050319A  3.240   20.20      Swift   3119
3    050401A  IBdHNe 050401A  2.900   92.00         KW   3179
4    050408A  IBdHNe 050408A  1.240   10.80       HETE   3188
5    050505A  IBdHNe 050505A  4.270   44.60      Swift   3364
6    050525A  IBdHNe 050525A  0.606    2.31         KW   3474
7    050730A  IBdHNe 050730A  3.970   42.80      Swift   3715
8    050802A  IBdHNe 050802A  1.710    7.46      Swift   3737
9    050814A  IBdHNe 050814A  5.300   26.80      Swift   3803
10   050820A  IBdHNe 050820A  2.610   39.00         KW   3852
11   050922C  IBdHNe 050922C  2.200   19.80         KW   4030
12   051109A  IBdHNe 051109A  2.350   18.30         KW   4238
13   060108A  IBdHNe 060108A  2.030    1.50      Swift   4445
14   060115A  IBdHNe 060115A  3.530   19.00      Swift   4518
15   060

## Swift GRBs table

In [8]:
grbstable = 'SwiftGRBsNASA_table.csv'
import os
if not os.path.exists(grbstable):
    with open(grbstable,'w') as fpout:
        import subprocess
        import shlex
        ret = subprocess.check_call( shlex.split('perl SwiftGRBsNASA.pl'), stdout=fpout, stderr=subprocess.STDOUT)

In [18]:
swf = pandas.read_csv(grbstable, sep=';')
print swf.sample(10)

          GRB Time|[UT] Trigger|Number BAT RA|(J2000) BAT Dec|(J2000)  \
130   151001B  18:29:36         657321        336.974          64.637   
120   151107A  17:19:36         662757        217.139         -59.680   
96    160220A  01:25:25         674670        236.942         -18.539   
1117   041223  14:06:18         100585        100.204         -37.068   
426   120811A  02:35:18         530581        257.184         -22.735   
911    070306  16:41:28         263361        148.154          10.440   
632   100302A  19:53:06         414592        195.504          74.568   
797    080605  23:47:57         313299        262.130           4.010   
209   141102A  12:51:40         617313        208.614         -47.100   
570   101117B  19:13:23         438675        172.991         -72.651   

     BAT 90%|Error Radius|[arcmin] BAT T90|[sec]  \
130                            2.1           109   
120                            2.7           n/a   
96                             1.8      

Regarding our goal -- to recover position and trigger time for each GRB -- we promptly see the `Time|[UT]` column, *RA*, *Dec* and the *error radius* given by `BAT`, `XRT` and `UVOT` instruments.

The *Time* column alone is incomplete since it does not inform the day the event happened; but this information is easily extracted from the GRB name itself.

Positional information comes in three flavors (*BAT*, *XRT*, *UVOT*), where *error radius* is the relevant information to decide which one to use; *BAT* has the worse precision (~`arcmin`) but has measurements for every GRB, while *XRT* and *UVOT* have much better measurements (~`arcsec`) but are followed less events.

By all means, let us work a little bit over the data in table itself.
For instance, let us handle the *Not Availables* (`n/a`) and then look some stats.

### GRB table pre-processing: fixing datatypes and finding nulls

In [173]:
# First, let's have a glance on columns values; let's try to figure out which one has "nulls".
#
# One way of doing this is by looking the frequency of values -- if a given value is two frequent, probably 
# that is the value is being used as 'NA'(i.e, Null) for that column (this comes with the assumption that such 
# table has a somewhat broad distribution of values);
#
# Before going further, what are columns datatypes?
print swf.dtypes

GRB                                              object
Time|[UT]                                        object
Trigger|Number                                   object
BAT RA|(J2000)                                   object
BAT Dec|(J2000)                                  object
BAT 90%|Error Radius|[arcmin]                    object
BAT T90|[sec]                                    object
BAT Fluence|(15-150 keV)|[10-7 erg/cm2]          object
XRT RA|(J2000)                                   object
XRT Dec|(J2000)                                  object
XRT 90%|Error Radius|[arcsec]                    object
XRT Time to First|Observation|[sec]              object
XRT Early Flux|(0.3-10 keV)|[10-11 erg/cm2/s]    object
UVOT RA|(J2000)                                  object
UVOT Dec|(J2000)                                 object
UVOT 90%|Error Radius|[arcsec]                   object
UVOT Time to|First Observation|[sec]             object
UVOT Magnitude                                  

In [175]:
# So, what are the frequency of values we find at each column?
#
# I am assuming that (1) few (correct) values are going to show up more than once, simply because we are dealing
# with real number and free-form comments; in principle, there is no restriction on the values.
# Following that, I assume that Null values will be among the most frequent values if not the only ones in the
# columns they occur.
freqs = pandas.DataFrame( {col:swf[col].value_counts().head().index.values for col in swf.columns} )
print freqs

  BAT 90%|Error Radius|[arcmin] BAT Dec|(J2000)  \
0                           1.0         -26.667   
1                           1.1         -44.177   
2                           1.2          13.039   
3                           1.8           9.139   
4                           1.7         -26.146   

  BAT Fluence|(15-150 keV)|[10-7 erg/cm2] BAT RA|(J2000) BAT T90|[sec]  \
0                                     n/a        241.273           n/a   
1                                      14        144.521            48   
2                                      13        282.338            64   
3                                      12        120.198          48.0   
4                                      11        270.992          11.0   

                                            Comments      GRB  \
0                                 UVOT: no detection  150821A   
1               XRT: no detection|UVOT: no detection   071129   
2  No AT slew (Earth limb constraint)|UVOT: no de... 

We see then that some columns do present Null values and they are all '**n/a**'.
We should be able to mask them very easily.

In [186]:
# Mask 'n/a' values
for col in swf.columns:
    swf[col].replace('n/a',None, inplace=True)
freqs = pandas.DataFrame( {col:swf[col].value_counts().head().index.values for col in swf.columns} )
print freqs

  BAT 90%|Error Radius|[arcmin] BAT Dec|(J2000)  \
0                           1.0          36.669   
1                           1.1         -44.177   
2                           1.2         -26.146   
3                           1.8          49.838   
4                           1.7           9.139   

  BAT Fluence|(15-150 keV)|[10-7 erg/cm2] BAT RA|(J2000) BAT T90|[sec]  \
0                                      13        312.599            48   
1                                      14        241.273            64   
2                                      12        355.412          48.0   
3                                      11        270.992          11.0   
4                                      15         92.387           7.6   

                                            Comments      GRB  \
0                                 UVOT: no detection  150821A   
1               XRT: no detection|UVOT: no detection   071129   
2  No AT slew (Earth limb constraint)|UVOT: no de... 

Nulls are properly flagged/masked now. This is important because we can now handle the data with confidence.

In [141]:
# (2) let's cast the columns (at least the ones we care about)
#
# Trigger time; we have to merge the information (Year-Month-Day) from column 'GRB' with UT time from 'Time'
import re
rec = re.compile('(\d+)?',re.IGNORECASE)
dt = pandas.DataFrame()
dt['day'] = swf.GRB.map(lambda s:'20{0}-{1}-{2}'.format(*re.findall('.{1,2}',rec.match(s).group(1))))
dt['time'] = swf['Time|[UT]']
print dt.head(10)

          day         time
0  2017-03-25  07:56:57.95
1  2017-03-18     15:27:52
2  2017-03-18     12:11:56
3  2017-03-17     09:45:59
4  2017-03-11     08:08:42
5  2017-03-07     20:24:21
6  2017-03-06     07:06:12
7  2017-02-08     22:33:38
8  2017-02-08     18:11:16
9  2017-02-06     21:57:25


In [147]:
dt['tt'] = pandas.to_datetime( dt.apply(lambda x:' '.join([x.day,x.time]), axis=1), yearfirst=1, errors='coerce')
# option 'coerce' enables the processing to complete even if some value was not able to be converted, in such cases
# 'NaT' ("null") is placed whenever convertion fails.
#
# That said, let's check whether the was any problem during conversion by verifying for Nulls
any(dt['tt'].isnull())

True

In [ ]:
# transform string 'na' values into pandas'
